# rubot Mecanum model description

This model has been extracted from: https://github.com/RBinsonB/nexus_4wd_mecanum_simulator


<img src="./Images/1_rubot_mecanum.png">

Run the simulation for one nexus robot by using the following command line:

In [ ]:
roslaunch nexus_4wd_mecanum_gazebo nexus_4wd_mecanum_world.launch

<img src="./Images/1_rubot_gazebo1.png">

To launch two nexus robots, use the following launch file:

In [ ]:
roslaunch nexus_4wd_mecanum_gazebo nexus_4wd_mecanum_multi.launch

<img src="./Images/1_rubot_gazebo2.png">

In [ ]:
sudo apt-get install ros-melodic-teleop-tools

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

# rUBot mecanum model
This robot includes a:
- 2D camera
- LIDAR

we have added the corresponding links and joints in "rubot_mecanum.xacro" file and the gazebo plugins in "rubot_mecanum.gazebo" file